In [1]:
import random
import pandas as pd
import numpy as np
import Covid19_model as CM
import matplotlib.pyplot as plt
import numpy as np
import shutil
import sys
import os.path
import math
import csv
from pyomo.environ import *
import lhsmdu
import time

# Set parameter

In [2]:
D = 1e2
n_site = 4
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 1e6

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(3)
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne, num_sample = 0, 0, 0, 20000
done = False
index = [0]
start_time = 6
for t in range(T):
    pset = env.para_sampling(100)
    para = pset[index[0]]
    action_buffer = [] 
    reward_buffer = []
    
    if budget > 0 and t > start_time:     
        for j in range(num_sample):  
            b0 = budget
            s0 = score
            ac = [random.random(),random.random(),random.random(),random.random(),random.random()]
            ac = [ac * n_site]
            ac = np.reshape(np.array(ac), (n_site, 5)).T
            
            action_buffer.append(ac) 
            state_ex, reward, _, _, c = env.transition(state, ac, para)  
            s0 += reward
            b0 -= c
            for i in range(t, T):
                if b0 > 0:
                    act = np.ones((5, n_site)) 
                    
                else:
                    act = np.zeros((5, n_site))
                next_ex, reward_ex, _, _, c = env.transition(state_ex, act, para)  
                s0 += reward_ex
                b0 -= c
                state_ex = next_ex.copy()          
            reward_buffer.append(s0)
               
        max_reward = np.argmax(np.array(reward_buffer))
        action = action_buffer[max_reward]
    else:
        action = np.zeros((5, n_site))
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, pset, 100, 1)

    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]), 
                  budget, score))
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]), 
                  np.sum(state[4]), np.sum(state[5]), budget, score))
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    print(action)
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 Time: 0 | S: 12563972.0, E: 28.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 1000000.00, reward:-57.32
 Time: 0 | S: 12563972.0, E: 28.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 1000000.00, reward:-57.32
 Time: 0 | new_e:13.2, cum_e:13.2, new_i:11.8, cum_i:11.8
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
 Time: 1 | S: 12563950.0, E: 37.0, A: 0.0, I: 11.8, D: 0.0, R: 1.3, Budget: 1000000.00, reward:-138.59
 Time: 1 | S: 12563949.0, E: 36.0, A: 0.0, I: 11.8, D: 0.0, R: 0.0, Budget: 1000000.00, reward:-138.59
 Time: 1 | new_e:18.6, cum_e:31.8, new_i:16.3, cum_i:28.2
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
 Time: 2 | S: 12563919.0, E: 49.9, A: 0.0, I: 28.1, D: 0.0, R: 2.9, Budget: 1000000.00, reward:-254.10
 Time: 2 | S: 12563919.0, E: 50.0, A: 0.0, I: 28.1, D: 0.0, R: 0.0, Budget: 1000000.00, reward:-254.10
 Time: 2 | new_e:26.2, cum_e:58.0, new_i:22.8, cum_i:51.0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]


 Time: 19 | S: 12435747.0, E: 162.0, A: 216.2, I: 630.0, D: 2.5, R: 127242.2, Budget: 866114.92, reward:-5149.89
 Time: 19 | S: 12435794.0, E: 156.0, A: 216.2, I: 630.0, D: 2.5, R: 127240.0, Budget: 866114.92, reward:-5149.89
 Time: 19 | new_e:88.9, cum_e:1150.6, new_i:99.8, cum_i:1307.1
[[0.00635821 0.00635821 0.00635821 0.00635821]
 [0.68760255 0.68760255 0.68760255 0.68760255]
 [0.92950827 0.92950827 0.92950827 0.92950827]
 [0.89801111 0.89801111 0.89801111 0.89801111]
 [0.0868395  0.0868395  0.0868395  0.0868395 ]]
 Time: 20 | S: 12436042.2, E: 214.3, A: 195.2, I: 675.4, D: 2.7, R: 126870.3, Budget: 839466.68, reward:-5593.83
 Time: 20 | S: 12436098.0, E: 205.0, A: 195.2, I: 675.4, D: 2.7, R: 126872.0, Budget: 839466.68, reward:-5593.83
 Time: 20 | new_e:101.2, cum_e:1251.7, new_i:111.3, cum_i:1418.4
[[0.77689807 0.77689807 0.77689807 0.77689807]
 [0.98635618 0.98635618 0.98635618 0.98635618]
 [0.99390176 0.99390176 0.99390176 0.99390176]
 [0.98899415 0.98899415 0.98899415 0.988994

 Time: 35 | S: 12235004.2, E: 108.7, A: 228.4, I: 828.2, D: 7.7, R: 327822.8, Budget: 659820.25, reward:-9370.72
 Time: 35 | S: 12235047.0, E: 112.0, A: 228.4, I: 828.2, D: 7.7, R: 327830.0, Budget: 659820.25, reward:-9370.72
 Time: 35 | new_e:50.7, cum_e:2054.2, new_i:69.9, cum_i:2595.3
[[0.82113053 0.82113053 0.82113053 0.82113053]
 [0.99940625 0.99940625 0.99940625 0.99940625]
 [0.99674956 0.99674956 0.99674956 0.99674956]
 [0.99328669 0.99328669 0.99328669 0.99328669]
 [0.89818607 0.89818607 0.89818607 0.89818607]]
 Time: 36 | S: 12203252.5, E: 60.1, A: 276.2, I: 794.6, D: 7.8, R: 359608.8, Budget: 655529.22, reward:-9538.06
 Time: 36 | S: 12203289.0, E: 63.0, A: 276.2, I: 794.6, D: 7.8, R: 359618.0, Budget: 655529.22, reward:-9538.06
 Time: 36 | new_e:28.3, cum_e:2082.5, new_i:54.9, cum_i:2650.2
[[0.13850816 0.13850816 0.13850816 0.13850816]
 [0.1678243  0.1678243  0.1678243  0.1678243 ]
 [0.9991006  0.9991006  0.9991006  0.9991006 ]
 [0.99105563 0.99105563 0.99105563 0.99105563]


 Time: 51 | S: 12090355.5, E: 68.9, A: 166.1, I: 762.8, D: 14.9, R: 472631.8, Budget: 477081.53, reward:-12581.47
 Time: 51 | S: 12090376.0, E: 73.0, A: 166.1, I: 762.8, D: 14.9, R: 472649.0, Budget: 477081.53, reward:-12581.47
 Time: 51 | new_e:33.9, cum_e:2695.3, new_i:47.8, cum_i:3545.4
[[0.99549394 0.99549394 0.99549394 0.99549394]
 [0.97424464 0.97424464 0.97424464 0.97424464]
 [0.99617838 0.99617838 0.99617838 0.99617838]
 [0.96516388 0.96516388 0.96516388 0.96516388]
 [0.97371977 0.97371977 0.97371977 0.97371977]]
 Time: 52 | S: 12052805.2, E: 35.4, A: 197.6, I: 717.5, D: 15.0, R: 510229.2, Budget: 476478.63, reward:-12712.79
 Time: 52 | S: 12052825.0, E: 36.0, A: 197.6, I: 717.5, D: 15.0, R: 510246.0, Budget: 476478.63, reward:-12712.79
 Time: 52 | new_e:23.0, cum_e:2718.3, new_i:40.3, cum_i:3585.7
[[0.04298696 0.04298696 0.04298696 0.04298696]
 [0.37300159 0.37300159 0.37300159 0.37300159]
 [0.99599947 0.99599947 0.99599947 0.99599947]
 [0.89871632 0.89871632 0.89871632 0.8987

 Time: 67 | S: 11930338.7, E: 63.4, A: 176.4, I: 556.8, D: 18.5, R: 632846.1, Budget: 302381.19, reward:-15191.94
 Time: 67 | S: 11930373.0, E: 62.0, A: 176.4, I: 556.8, D: 18.5, R: 632858.0, Budget: 302381.19, reward:-15191.94
 Time: 67 | new_e:30.1, cum_e:3248.1, new_i:45.8, cum_i:4301.9
[[0.65266187 0.65266187 0.65266187 0.65266187]
 [0.85750996 0.85750996 0.85750996 0.85750996]
 [0.98861728 0.98861728 0.98861728 0.98861728]
 [0.98342464 0.98342464 0.98342464 0.98342464]
 [0.95581812 0.95581812 0.95581812 0.95581812]]
 Time: 68 | S: 11907849.5, E: 32.5, A: 200.8, I: 542.8, D: 18.6, R: 655355.8, Budget: 298202.80, reward:-15280.47
 Time: 68 | S: 11907884.0, E: 32.0, A: 200.8, I: 542.8, D: 18.6, R: 655369.0, Budget: 298202.80, reward:-15280.47
 Time: 68 | new_e:17.4, cum_e:3265.5, new_i:36.6, cum_i:4338.4
[[0.1836061  0.1836061  0.1836061  0.1836061 ]
 [0.70970025 0.70970025 0.70970025 0.70970025]
 [0.97042552 0.97042552 0.97042552 0.97042552]
 [0.99613781 0.99613781 0.99613781 0.9961

 Time: 83 | S: 11808881.4, E: 28.4, A: 103.3, I: 386.2, D: 21.2, R: 754579.5, Budget: 150008.45, reward:-16658.13
 Time: 83 | S: 11808926.0, E: 28.0, A: 103.3, I: 386.2, D: 21.2, R: 754592.0, Budget: 150008.45, reward:-16658.13
 Time: 83 | new_e:15.1, cum_e:3548.7, new_i:24.1, cum_i:4788.2
[[0.00471658 0.00471658 0.00471658 0.00471658]
 [0.79249006 0.79249006 0.79249006 0.79249006]
 [0.98532204 0.98532204 0.98532204 0.98532204]
 [0.94868892 0.94868892 0.94868892 0.94868892]
 [0.52150605 0.52150605 0.52150605 0.52150605]]
 Time: 84 | S: 11812862.8, E: 26.6, A: 100.1, I: 372.0, D: 21.3, R: 750617.3, Budget: 149066.10, reward:-16715.81
 Time: 84 | S: 11812908.0, E: 26.0, A: 100.1, I: 372.0, D: 21.3, R: 750629.0, Budget: 149066.10, reward:-16715.81
 Time: 84 | new_e:12.4, cum_e:3561.1, new_i:21.8, cum_i:4810.1
[[0.04429181 0.04429181 0.04429181 0.04429181]
 [0.94250548 0.94250548 0.94250548 0.94250548]
 [0.99366537 0.99366537 0.99366537 0.99366537]
 [0.99069885 0.99069885 0.99069885 0.9906

In [2]:
5935.49777317047 / 83

71.51202136349964